<a href="https://colab.research.google.com/github/FreddyGG/TP_BIG_DATA/blob/main/Trabajo_Pr%C3%A1ctico_de_Big_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **COMPONENTE 1**

Como paso inicial del proyecto de big data, el alumno escribir un script que permita obtener una
lista de las últimas menciones de un handle o hashtag en Twitter a través de su API. Este
programa en Python deberá luego guardar los tweets en un archivo .csv para su posterior
análisis.
El archivo de salida deberá contener mínimamente: Texto del tweet, Autor, Timestamp, Número de Favs, Número de RTs

In [29]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [30]:
!pip install langdetect

In [31]:
#importamos librerias
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import datetime
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import SnowballStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words('spanish'))
from sklearn.feature_extraction.text import CountVectorizer
from langdetect import detect

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
# Twitter Api Credentials
consumerKey = 'ogYNVLjPL8ygj4PO6vIyYVxyP'#log["key"][0]
consumerSecret = 'fS8uxKMb2hZ4GOpey1Y6AsbWDeQCIA9srJRetzCo9cAMAM6AbZ'#log["key"][1]
accessToken = '175984347-syxFReYiggzkhkr0gz5bKX4Asi5S91eNXKU0iIbV'#log["key"][2]
accessTokenSecret = '1LhUQr7jCEDxZ0BBQSs9TK2VaXiZq08f2pKMMoBPq1GkJ'#log["key"][3]

In [33]:
#creamos un objeto de autenticacion
authenticate = tweepy.OAuthHandler(consumerKey, consumerSecret) 
    
# Seteamos el access token y access token secret
authenticate.set_access_token(accessToken, accessTokenSecret) 

#Creamos el objeto API mientras parseamos la informacion en auth  
api = tweepy.API(authenticate, wait_on_rate_limit = True)

In [35]:
#Pedimos que ingrese los tweets a buscar
print('Analizar Sentimientos de Tweets')
keyword = input("Ingrese el hastag o keyword que desea buscar: ")
nroTweet = int(input("Ingrese cuantos tweets desea buscar: "))

list = tweepy.Cursor(api.search, q=keyword ,tweet_mode='extended', lang='es').items(nroTweet)

Analizar Sentimientos de Tweets
Ingrese el hastag o keyword que desea buscar: Energia Solar
Ingrese cuantos tweets desea buscar: 500


In [36]:
#Almacenamos los tweets en una lista
tweet_list = []
for tweet in list:
    text = tweet._json["full_text"]
    #print(text)
    favourite_count = tweet.favorite_count
    retweet_count = tweet.retweet_count
    created_at = tweet.created_at
    author = tweet.author.name
    
    line = {'text' : text, 'author' : author, 'favourite_count' : favourite_count, 'retweet_count' : retweet_count, 'created_at' : created_at}
    tweet_list.append(line)

In [37]:
#Convertimos la lista a un DataFrame
dfTweets = pd.DataFrame(tweet_list)

#Seteamos la funcion para limpiar los tweets
def cleanTxt(text):
 text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
 text = re.sub('#', '', text) # Removing '#' hash tag
 text = re.sub('RT[\s]+', '', text) # Removing RT
 text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
 
 return text

#Limpiamos los tweets y renombramos las columnas
dfTweets = dfTweets.rename(columns={'text': 'Tweets', 'author': 'Author', 'favourite_count': 'Favourites', 'retweet_count': 'Retweets', 'created_at': 'Date'})
dfTweets['Tweets'] = dfTweets['Tweets'].apply(cleanTxt)

#Creamos el archivo csv
dfTweets.to_csv('tweets.csv')

In [38]:
dfTweets

,Tweets,Author,Favourites,Retweets,Date
0,: Gracias por este premio Nacional de Economía...,mistigri12@hotmail.c,0,29,2021-06-23 19:18:45
1,: Ciudad Solar es uno de los ejes de Nuestro P...,Abel Mandujano,0,108,2021-06-23 19:18:20
2,: Ciudad Solar es uno de los ejes de Nuestro P...,Medea,0,108,2021-06-23 19:18:00
3,_Villeda: Los costos de la energía solar fotov...,CuidadaniaQro,0,70,2021-06-23 19:17:52
4,: Ciudad Solar es uno de los ejes de Nuestro P...,JM Gomez,0,108,2021-06-23 19:17:00
...,...,...,...,...,...
495,: Subsecretario de Energía inaugura proyecto ...,lomejorchilito,0,14,2021-06-23 12:47:23
496,: Subsecretario de Energía inaugura proyecto ...,Energía Ñuble,0,14,2021-06-23 12:47:16
497,: Subsecretario de Energía inaugura proyecto ...,Energía Los Lagos,0,14,2021-06-23 12:47:10
498,: Lugares en Adjuntas urbano que gozan de ener...,CoHemis,0,98,2021-06-23 12:46:59


# **COMPONENTE 2**

El alumno deberá crear un script en Python que calcule el sentimiento de cada tweet y
almacene los resultados en un archivo .csv. Además, se deberá calcular el sentimiento
promedio relacionado a las menciones.

In [39]:
#creamos un data frame con unicamente los tweets
tweets = pd.DataFrame(dfTweets['Tweets'])

In [40]:
tweets.head()

,Tweets
0,: Gracias por este premio Nacional de Economía...
1,: Ciudad Solar es uno de los ejes de Nuestro P...
2,: Ciudad Solar es uno de los ejes de Nuestro P...
3,_Villeda: Los costos de la energía solar fotov...
4,: Ciudad Solar es uno de los ejes de Nuestro P...


In [41]:
# crear la funcion para obtener la subjectividad
def getSubjectivity(text):
   return TextBlob(text).sentiment.subjectivity

# Crear la funcion para obtener la polaridad
def getPolarity(text):
   return  TextBlob(text).sentiment.polarity


# Crear dos nuevas columnas 'Subjectividad' & 'Polaridad'
tweets['Subjectividad'] = tweets['Tweets'].apply(getSubjectivity)
tweets['Polaridad'] = tweets['Tweets'].apply(getPolarity)

# Show the new dataframe with columns 'Subjectivity' & 'Polarity'
tweets

,Tweets,Subjectividad,Polaridad
0,: Gracias por este premio Nacional de Economía...,0.0,0.0
1,: Ciudad Solar es uno de los ejes de Nuestro P...,0.0,0.0
2,: Ciudad Solar es uno de los ejes de Nuestro P...,0.0,0.0
3,_Villeda: Los costos de la energía solar fotov...,0.0,0.0
4,: Ciudad Solar es uno de los ejes de Nuestro P...,0.0,0.0
...,...,...,...
495,: Subsecretario de Energía inaugura proyecto ...,0.0,0.0
496,: Subsecretario de Energía inaugura proyecto ...,0.0,0.0
497,: Subsecretario de Energía inaugura proyecto ...,0.0,0.0
498,: Lugares en Adjuntas urbano que gozan de ener...,0.0,0.0


In [48]:
# Crear una funcion para calcular negativo (-1), neutral (0) and positivo (+1) 
def getAnalysis(score):
  if score < 0:
    return 'Negativo'
  elif score == 0:
    return 'Neutral'
  else:
    return 'Positivo'

tweets['Analisis'] = tweets['Polaridad'].apply(getAnalysis)

#Convertimos el df en csv
tweets.to_csv('TweetsSentimentAnalysis.csv')

# Show the dataframe
tweets

,Tweets,Subjectividad,Polaridad,Analisis
0,: Gracias por este premio Nacional de Economía...,0.0,0.0,Neutral
1,: Ciudad Solar es uno de los ejes de Nuestro P...,0.0,0.0,Neutral
2,: Ciudad Solar es uno de los ejes de Nuestro P...,0.0,0.0,Neutral
3,_Villeda: Los costos de la energía solar fotov...,0.0,0.0,Neutral
4,: Ciudad Solar es uno de los ejes de Nuestro P...,0.0,0.0,Neutral
...,...,...,...,...
495,: Subsecretario de Energía inaugura proyecto ...,0.0,0.0,Neutral
496,: Subsecretario de Energía inaugura proyecto ...,0.0,0.0,Neutral
497,: Subsecretario de Energía inaugura proyecto ...,0.0,0.0,Neutral
498,: Lugares en Adjuntas urbano que gozan de ener...,0.0,0.0,Neutral


In [49]:
#Porcentaje de Tweets Positivos
ntweets = tweets[tweets.Analisis == 'Positivo']
ntweets = ntweets['Tweets']
ntweets

round( (ntweets.shape[0] / tweets.shape[0]) * 100, 1)

1.0

In [50]:
#Porcentaje de Tweets Negativos
# Print the percentage of negative tweets
ntweets = tweets[tweets.Analisis == 'Negativo']
ntweets = ntweets['Tweets']
ntweets

round( (ntweets.shape[0] / tweets.shape[0]) * 100, 1)

0.2

In [51]:
#Porcentaje de Tweets Neutros
ntweets = tweets[tweets.Analisis == 'Neutral']
ntweets = ntweets['Tweets']
ntweets

round( (ntweets.shape[0] / tweets.shape[0]) * 100, 1)

98.8